In [1]:
import tensorflow_probability as tfp
import numpy as np
import tensorflow as tf

import time
from tqdm import tqdm
import pickle

from math import *

import matplotlib.dates as mdates
import matplotlib.ticker as ticker

myFmt = mdates.DateFormatter('%Hh')

import pandas as pd
np.set_printoptions(suppress=True,precision=6)
import matplotlib.pyplot as plt
import matplotlib

plt.style.use('ggplot') 
plt.style.use('seaborn-paper') 
plt.style.use('seaborn-whitegrid')

import sys

from tensorflow_probability import distributions as tfd
from geopy.distance import geodesic

%matplotlib inline

#from move_ns import moveNS
from move_ns_whiten import moveNS


## Load data and display individual time series lengths

In [2]:
# set up lower level GP locations covering 24 hours
Z = np.linspace(0,24,num=25,endpoint=False).astype(np.float64)[:,None]
#np.random.shuffle(Z)
#Z = np.random.uniform(0,24,size=(25,1)).astype(np.float64)
#Z.shape

In [3]:
def setup_data(skip_i=1,skip=3):
    
    df = pd.read_csv('data/ovejas.csv')

    df = df[df.id!=34]
    
    



    df['ID'] = df['id'].astype('category').cat.rename_categories(range(0, df['id'].nunique())).astype('int')
    df = df[df['ID']%skip_i==0]
    
    ID = df['ID'].values 
    
    
    
    Xgps = df[['lat','lon']].values
    minX = np.min(Xgps[:,0])
    minY = np.min(Xgps[:,1])




    secs =(pd.to_datetime(df['time'])- pd.datetime(2018,1,1)).dt.seconds.astype(float).values
    days = (pd.to_datetime(df['time'])- pd.datetime(2018,1,1)).dt.days.astype(float).values
    T = (days*24*60+secs/60)/(60*24) #days
    T = T-np.min(T)

    rescale = 24  # use hours to improve numerical stability
    T = T * rescale


    




    # use geodesic to get the straight line distance between two points
    Xmetres = np.array([geodesic((xloc,minY), (minX,minY)).meters for xloc in Xgps[:,0]])
    Ymetres = np.array([geodesic((minX,yloc), (minX,minY)).meters for yloc in Xgps[:,1]])

    


    X = np.array([Xmetres, Ymetres]).T
    
    T=T[::skip,None]
    X=X[::skip]
    ID=IDs[::skip]

    return X, T, ID


In [4]:
X,T,ID = setup_data(skip_i=1,skip=2)
X[:,0] = X[:,0]-X[:,0].mean()
X[:,1] = X[:,1]-X[:,1].mean()
X[:,0] = X[:,0]/1000#X[:,1].std()
X[:,1] = X[:,1]/1000#X[:,1].std()

print(X.shape)

<ipython-input-3-3f542ce23d13>:27: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  secs =(pd.to_datetime(df['time'])- pd.datetime(2018,1,1)).dt.seconds.astype(float).values
<ipython-input-3-3f542ce23d13>:28: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  days = (pd.to_datetime(df['time'])- pd.datetime(2018,1,1)).dt.days.astype(float).values


processing 0
processing 1
processing 2
processing 3
processing 4
processing 5
processing 6
processing 7
processing 8
processing 9
processing 10
processing 11
processing 12
processing 13
processing 14
processing 15
processing 16
processing 17
processing 18
processing 19
processing 20
processing 21
processing 22
processing 23
processing 24
processing 25
processing 26
(225953, 2)


In [5]:
def sp_shift(x):
    # softplus transform with shift 
    return tf.nn.softplus(x)+1e-4

## Set-up the non-stationary GP

In [6]:


def periodic_kernel(x1,x2):
    # periodic kernel with parameter set to encode
    # daily activity pattern (period=rescale).
    x2a = x2+ np.float64(1)#2*0.18)#24/pi)
    return tfp.math.psd_kernels.ExpSinSquared(x1,x2a,np.float64(24.0))

# transform for parameter to ensure positive
transforms=[sp_shift,sp_shift] 
#transforms=[sp_shift] 

# diffuse priors on parameters
lpriors = [tfd.Normal(loc = np.float64(0.),scale=np.float64(1)),
           tfd.Normal(loc = np.float64(0),scale=np.float64(1)),
           tfd.Normal(loc = np.float64(0),scale=np.float64(10.))]
           
apriors = [tfd.Normal(loc = np.float64(0.),scale=np.float64(1)),
           tfd.Normal(loc = np.float64(0.),scale=np.float64(1)),
           tfd.Normal(loc = np.float64(0),scale=np.float64(10.))]


lparams_init = [0.0,0.0,0.0]#p.mean().numpy() for p in lpriors]
aparams_init = [0.0,0.0,0.0]#[0.0 for p in apriors]


# create the model #2880
mover = moveNS(T,X,Z, ID, BATCH_SIZE=1000, MIN_REMAIN=500,velocity=True, std_obs_noise=100, mean_obs_noise=10,
                        akernel=periodic_kernel, 
                        aparams_init=aparams_init, 
                        apriors=apriors, 
                        atransforms=transforms,
                        lkernel=periodic_kernel, 
                        lparams_init=lparams_init, 
                        lpriors=lpriors, 
                        ltransforms=transforms)





In [ ]:

learning_rate = tf.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-1,
    decay_steps=10,
    decay_rate=0.99,
    staircase=True)


optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate,beta_2=0.99)
train_steps = 2000
pbar = tqdm(range(train_steps))
loss_history = np.zeros((train_steps))
for i in pbar:
    with tf.GradientTape() as t:
        loss = -mover.log_posterior(*mover.kernel_params)
    loss_history[i] = loss.numpy()
    pbar.set_description("Loss %f" % loss_history[i])

    gradients = t.gradient(loss, mover.kernel_params)
    optimizer.apply_gradients(zip(gradients, mover.kernel_params))
#n=3.5


In [ ]:
plt.plot(loss_history[1500:])
plt.show()
loss.numpy()

In [ ]:
lengths = mover.get_lengthscale()
plt.plot(Z,lengths,'o',c='b')
Zin = np.linspace(0,24,num=500,endpoint=False).astype(np.float64)[:,None]

lengths = mover.get_lengthscale(X=Zin)
plt.plot(Zin,lengths)
plt.show()

amps = mover.get_amplitude()
plt.plot(Z,amps,'o',c='b')
Zin = np.linspace(0,24,num=500,endpoint=False).astype(np.float64)[:,None]
amps = mover.get_amplitude(X=Zin)
plt.plot(Zin,amps)
plt.show()



In [ ]:
plt.plot(Z,(mover.kernel_params[2]).numpy(),'-o')
#plt.ylim(0,0.5)
plt.show()
plt.plot(Z,(mover.kernel_params[6]).numpy(),'-o')
plt.show()
#6

In [ ]:


opt_step = []
for var in optimizer.variables():
    if '/v:' in var.name:
        opt_step.append((np.sqrt((var.numpy()))+1e-16)**-1)
        
        
plt.plot(opt_step[2][0:25],'.')
plt.plot(opt_step[6][0:25],'.')
#plt.ylim(-0.5,1)
#plt.show()

for o in opt_step:
    print(o.dtype)

In [8]:
for i in range(len(opt_step)):
    o=opt_step[i]
    if len(o.shape):
        o[o>1]=1
    opt_step[i]=o#opt_step[i]*0+1e-2
    print(opt_step[i])

0.004842485281937078
0.012115679242881108
[0.005525 0.012081 0.007938 0.028228 0.008822 0.097371 0.010387 0.01392
 0.015413 0.011206 0.013048 0.016353 0.018095 0.024215 0.036783 0.054245
 0.088079 0.165327 0.339472 0.81622  1.       1.       1.       1.
 1.      ]
0.03458662682408681
0.027518707273064325
0.002804763469339627
[0.003491 0.006215 0.005549 0.011827 0.006601 0.073222 0.007797 0.01013
 0.01003  0.006337 0.006785 0.007145 0.006939 0.008356 0.010563 0.01337
 0.019494 0.031508 0.055949 0.116227 0.289064 0.868092 1.       1.
 1.      ]
0.0054500266418088675
0.09960409488610739


In [ ]:
for j, param in enumerate(mover.kernel_params):
    print(j,(param.numpy()),param.shape)

In [ ]:
tf.nn.softplus(-1.5)

In [ ]:
opt_params = [i.numpy() for i in  mover.kernel_params]
opt_params.append(Z)

with open('opt_params.npy', 'wb') as fp:
    pickle.dump(opt_params, fp)
with open('opt_step.npy', 'wb') as fp:
    pickle.dump(opt_step, fp)

In [7]:

with open ('opt_params.npy', 'rb') as fp:
    opt_params = pickle.load(fp)
    opt_obs_noise = opt_params[0]
    opt_ls_mean = opt_params[1]
    opt_ls_v = opt_params[2]
    opt_ls_amp = sp_shift(opt_params[3]).numpy()
    opt_ls_ls = sp_shift(opt_params[4]).numpy()
    opt_amp_mean = opt_params[5]
    opt_amp_v = opt_params[6]
    opt_amp_amp = sp_shift(opt_params[7]).numpy()
    opt_amp_ls = sp_shift(opt_params[8]).numpy()
    

    for i in range(len(mover.kernel_params)):
         mover.kernel_params[i].assign(opt_params[i])
    #opt_step = opt_params[9]
    Z = opt_params[9]

    
with open ('opt_step.npy', 'rb') as fp:
    opt_step = pickle.load(fp)
#mover.log_posterior(*mover.kernel_params)


In [ ]:
mover.log_posterior(*mover.kernel_params)


In [ ]:


def ls_periodic_kernel():
    # periodic kernel with single variable parameter. Other parameters are set 
    # to encode daily activity pattern (period=rescale).
    # 15 minute correlation time
    return tfp.math.psd_kernels.ExpSinSquared(np.float64(opt_ls_amp),np.float64(opt_ls_ls)+np.float64(1.0),np.float64(24.0))

def amp_periodic_kernel():
    # periodic kernel with single variable parameter. Other parameters are set 
    # to encode daily activity pattern (period=rescale).
    # 15 minute correlation time
    return tfp.math.psd_kernels.ExpSinSquared(np.float64(opt_amp_amp),np.float64(opt_amp_ls)+np.float64(1.0),np.float64(24.0))


# transform for parameter to ensure positive
transforms=[] 

# prior distribution on parameters - changed to 20 
lpriors = [tfd.Normal(loc = np.float64(opt_ls_mean),scale=np.float64(0.10))]
apriors = [tfd.Normal(loc = np.float64(opt_amp_mean),scale=np.float64(0.10))]

# random initial values of mean and kernel amplitude
lparams_init = [opt_ls_mean]
aparams_init = [opt_amp_mean]

# create the model 


mover_hmc = moveNS(T,X,Z, ID, BATCH_SIZE=1000, MIN_REMAIN= 500, velocity=True, std_obs_noise=0, mean_obs_noise=opt_obs_noise,
                        akernel=amp_periodic_kernel, 
                        aparams_init=aparams_init, 
                        apriors=apriors, 
                        atransforms=transforms,
                        lkernel=ls_periodic_kernel, 
                        lparams_init=lparams_init, 
                        lpriors=lpriors, 
                        ltransforms=transforms)

mover_hmc.kernel_params[1].assign(opt_ls_v)
mover_hmc.kernel_params[3].assign(opt_amp_v)


In [ ]:
3098/60

In [ ]:
print(end - start)


In [ ]:
plt.plot(s2,'-o')

In [ ]:
plt.plot(s1)

In [ ]:
# # sample from the posterior
kr = mover.hmc_sample(num_samples=1, skip=0, burn_in=0, num_leapfrog_steps=1, init_step=1e-8)
# lengths = mover.get_lengthscale_samples()
# amps = mover.get_amplitude_samples()
# #  plot the sample for checking
# plt.figure(figsize=(10,6))
# Z_time=np.array([np.datetime64('2019') + np.timedelta64(int(k*60*60), 's') for k in Z])

# for i in range(0,lengths.shape[0]):
#     plt.plot(Z_time[np.argsort(Z_time)],lengths[i,np.argsort(Z_time)],c='C1',alpha=0.902,linewidth=1.0)

# ax = plt.gca()
# ax.xaxis_date()
# ax.xaxis.set_major_formatter(myFmt)
# plt.figure(figsize=(10,6))

# for i in range(0,amps.shape[0]):
#     plt.plot(Z_time[np.argsort(Z_time)],amps[i,np.argsort(Z_time)],c='C1', alpha=0.902,linewidth=1.0)
# ax = plt.gca()
# ax.xaxis_date()
# ax.xaxis.set_major_formatter(myFmt)

In [ ]:

ss = np.float64(1e-2)
steps = []
for j, param in enumerate(mover.kernel_params):
    steps.append(ss)

# steps = []
# for j, param in enumerate(mover.kernel_params):
#     if j==2:
#         steps.append(ss)
#     elif j==6:
#         steps.append(ss)
#     else:
#         steps.append(0)
start = time.time()

num_samples=10#500
burn=10
start = time.time()
mover.num_samples = num_samples
#kr = mover.nuts_sample(num_samples=num_samples, skip=0, burn_in=burn, init_step=steps)

end = time.time()
print(i,end - start)


In [ ]:
inner_kernel = tfp.mcmc.NoUTurnSampler(target_log_prob_fn=mover.log_posterior,max_tree_depth=4, step_size=opt_step)
kernel = tfp.mcmc.DualAveragingStepSizeAdaptation(inner_kernel, num_adaptation_steps=int(burn * 0.8))
start = time.time()

samples, kernel_results = tfp.mcmc.sample_chain(num_results=num_samples, num_burnin_steps=burn, num_steps_between_results=0, current_state=mover.kernel_params, kernel=kernel)
end = time.time()
print(i,end - start)
mover.samples_ = samples
print(np.sum(kernel_results.inner_results.is_accepted.numpy()/num_samples))


In [ ]:
1 108.34
2 295
3 638
4 1322

In [ ]:
kernel_results.new_step_size

In [ ]:
# steps = []
# for j, param in enumerate(mover.kernel_params):
#     print(j,mover.kernel_params[j].shape)
#     if j==2:
#         steps.append(10*s1)
#     elif j==6:
#         steps.append(10*s2)
#     else:
#         steps.append(0)

In [ ]:
start = time.time()
# 200/200 took 6.31 - run for 1000/1000 = 31.551hrs
num_samples=250#00
burn_in=100#500  ## 100/250
num_runs=1#4
rescale = 24

for i in range(num_runs):
    ss = np.float64(1e-3)
    steps = []
    for j, param in enumerate(mover.kernel_params):
        steps.append(ss)
    steps=opt_step
    kr = mover.nuts_sample(num_samples=num_samples, skip=0, burn_in=burn_in, max_tree_depth=5, init_step=steps)
   
    # save the results
    final_hmc_np = [j.numpy() for j in  mover.samples_]
    with open('data/hmc_samples_p_' + str(i) + '.npy', 'wb') as fp:
        pickle.dump(final_hmc_np, fp)
        
    lengths = mover.get_lengthscale_samples()
    amps = mover.get_amplitude_samples()
    np.save('data/len_sheep_p_' + str(i) + '.npy',lengths)
    np.save('data/amp_sheep_p_' + str(i) + '.npy',amps)
    

    Zin = np.linspace(0,24,num=200,endpoint=False).astype(np.float64)[:,None]

    # sample from the posterior)
    lengths = mover.get_lengthscale_samples(X=Zin)
    amps = mover.get_amplitude_samples(X=Zin)

    np.save('data/full_len_sheep_p_' + str(i) + '.npy',lengths)
    np.save('data/full_amp_sheep_p_' + str(i) + '.npy',amps)
    np.save('data/Z_pred_p_' + str(i) + '.npy',Zin)

    
    end = time.time()
    print(i,end - start)

/home/staff1/ctorney/anaconda3/envs/tf/lib/python3.8/site-packages/tensorflow_probability/python/mcmc/sample.py:333: UserWarning: Tracing all kernel results by default is deprecated. Set the `trace_fn` argument to None (the future default value) or an explicit callback that traces the values you are interested in.
  warnings.warn("Tracing all kernel results by default is deprecated. Set "


In [ ]:
kr.new_step_size

In [ ]:

start = time.time()
# 200/200 took 6.31 - run for 1000/1000 = 31.551hrs
num_samples=1500
burn_in=500
num_runs=1
rescale = 24

for i in range(num_runs):
    steps = []
    
    ss = np.float64(1e-1)
    
    for j, param in enumerate(mover_hmc.kernel_params):
        if j==1:
            steps.append(s1)
        elif j==3:
            steps.append(s2)
        else:
            steps.append(0)
    kr = mover_hmc.hmc_sample(num_samples=num_samples, skip=0, burn_in=burn_in, num_leapfrog_steps=5, init_step=steps)
   
    # save the results
    final_hmc_np = [j.numpy() for j in  mover_hmc.samples_]
    with open('data/hmc_samples_p_' + str(i) + '.npy', 'wb') as fp:
        pickle.dump(final_hmc_np, fp)
        
    lengths = mover_hmc.get_lengthscale_samples()
    amps = mover_hmc.get_amplitude_samples()
    np.save('data/len_sheep_p_' + str(i) + '.npy',lengths)
    np.save('data/amp_sheep_p_' + str(i) + '.npy',amps)
    

    Zin = np.linspace(0,24,num=200,endpoint=False).astype(np.float64)[:,None]

    # sample from the posterior)
    lengths = mover_hmc.get_lengthscale_samples(X=Zin)
    amps = mover_hmc.get_amplitude_samples(X=Zin)

    np.save('data/full_len_sheep_p_' + str(i) + '.npy',lengths)
    np.save('data/full_amp_sheep_p_' + str(i) + '.npy',amps)
    np.save('data/Z_pred_p_' + str(i) + '.npy',Zin)

    
    end = time.time()
    print(i,end - start)

In [ ]:
print('time: ',10*(end - start)/60/60)

print(np.sum(kernel_results.inner_results.is_accepted.numpy()/num_samples))


In [ ]:
kr.new_step_size


In [ ]:
25.4872/5*6

In [ ]:
for mp in mover.kernel_params:
    print(mp)
    
    

In [ ]:
rescale = 24
Z_pred = np.arange(0,1.0*rescale,rescale*0.001).astype(np.float64)[:,None]
lengths = mover_hmc.get_lengthscale_samples(X=Z_pred)
amps = mover_hmc.get_amplitude_samples(X=Z_pred)

In [ ]:

Zin = np.linspace(0,24,num=500,endpoint=False).astype(np.float64)[:,None]

# sample from the posterior)
lengths = mover.get_lengthscale_samples(X=Zin)
amps = mover.get_amplitude_samples(X=Zin)


In [ ]:
#  plot the sample for checking
plt.figure(figsize=(10,6))


Z_time=np.array([np.datetime64('2019') + np.timedelta64(int(k*60*60), 's') for k in Zin])

for i in range(0,lengths.shape[0]):
    plt.plot(Z_time[np.argsort(Z_time)],lengths[i,np.argsort(Z_time)],c='C1',alpha=0.902,linewidth=1.0)

ax = plt.gca()
ax.xaxis_date()
ax.xaxis.set_major_formatter(myFmt)
plt.figure(figsize=(10,6))

for i in range(0,amps.shape[0]):
    plt.plot(Z_time[np.argsort(Z_time)],amps[i,np.argsort(Z_time)],c='C1', alpha=0.902,linewidth=1.0)
ax = plt.gca()
ax.xaxis_date()
ax.xaxis.set_major_formatter(myFmt)